Code Dependencies
    - envoy
    - progressbar
    - sklearn 0.17
    - Cython
 

In [58]:
from lrec.utils.data_utils.data import Data, loadDataset
from lrec.utils.data_utils.lineParser import UserItemRatingParser

In [ ]:
train_path = TRAIN_DATA_PATH
test_path = TEST_DATA_PATH

<h2>Load Data</h2>

#### Input data format:
    <user_id>\t<item_id>\t<rating>
    Note: "rating" is 1 for one class collaborative filtering

In [84]:
#Define the dataset parser
parser = UserItemRatingParser(delim="\t")
d = Data()
d.import_data(train_path, parser)
train = d.R
test, _ = loadDataset(test_path, d.users, d.items, parser)

<h2>Learn LRec Model</h2>

<h3>Learn model in parallel</h3>
Start ipython parallel engines.

For eg: start 4 ipython engines

ipcluster start -n 4

Loss Functions
   - squared : squared loss 
   - squared_analytical : squared loss solved analytically. Preferred when user-user matrix can fit into the memory
   - logistic: logistic loss

In [82]:
from lrec.recommender.LRec.LRec import LRec
from lrec.recommender.modelArgs import LRecArgs
from lrec.evaluate.eval_ranking_metric import evalMetricsParallelMiniBatch

#number of cpu to use
n = 4
#set loss function (squared or logisitc)
loss = "logistic"
#set regularization strength
c = 0.0001
arg = LRecArgs(c, loss)
model = LRec(arg)
indices, sim = model.fit_parallel(train,num_procs=n)

<h2>Evaluate the model</h2>

In [83]:
from lrec.evaluate.eval_ranking_metric import evalMetricsParallelMiniBatch
evalMetricsParallelMiniBatch(train, train, test, model, mapk=100, ks=[3, 5, 10, 20])